In [1]:
import csv
import itertools
import sys

In [2]:
csvs = ["demo.csv", "demo2.csv"]

In [3]:
samples = [csv.split(".")[0] for csv in csvs]
samples

['demo', 'demo2']

In [4]:
rank = "species"
lineage_dict = {}
sample_name_dict = {}
seen_lineages = set()

# create dictionary that holds all of the sample names
for file in csvs:
    sample_name = file.split('.')[0]
    sample_name_dict[sample_name] = 0 

for file in csvs:
    with open(file, 'r') as fp:
        r = csv.DictReader(fp)
        for n, row in enumerate(r):
            if row["rank"] == rank:
                seen_lineages.add(row["lineage"])
        fp.close()

for lineage in seen_lineages:
    lineage_dict[lineage] = sample_name_dict.copy()

for sample in sample_name_dict:
    with open(sample + ".csv", "r") as fp:
        r = csv.DictReader(fp)
        for n, row in enumerate(r):
            if row["rank"] == rank:
                lineage = (row["lineage"])
                fraction = (row["fraction"])
                lineage_dict[lineage][sample] = fraction
        fp.close()


print(lineage_dict)

{'d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella copri': {'demo': '0.057', 'demo2': '0.057'}, 'd__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Phocaeicola;s__Phocaeicola vulgatus': {'demo': '0.016', 'demo2': 0}, 'd__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia;s__Escherichia coli': {'demo': '0.058', 'demo2': '0.058'}, 'd__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Phocaeicola;s__Fake species': {'demo': 0, 'demo2': '0.016'}}


In [5]:
samples.insert(0, "lineage")

w = csv.DictWriter( sys.stdout, samples)
for key,val in sorted(lineage_dict.items()):
    row = {'lineage': key}
    row.update(val)
    w.writerow(row)

d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Phocaeicola;s__Fake species,0,0.016
d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Phocaeicola;s__Phocaeicola vulgatus,0.016,0
d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella copri,0.057,0.057
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia;s__Escherichia coli,0.058,0.058


In [15]:
def format_tax_to_frac(taxonomy_csvs, rank, output_csv):
    '''
    takes the output for sourmash taxonomy summarize and produces a 
    tab-separated file with fractions for each sample. Sample names
    are based on csv file names, with ".csv" removed
    lineage	sample1	sample2	sample3	
    lin_a	.4	.17	.6
    lin_b	0	0	.1
    lin_c	0	.3	0
    lin_d	.2	.1	0
    lin_e	0	0	.01
    lin_f	0	.07	0
    lin_g	0	0	0
    lin_h	.3	.4	.2
    '''
    samples = [csv.split(".")[0] for csv in csvs]

    possible_ranks = ['superkingdom', "phylum", "class", "order", "family", "genus", "species"]
    if rank not in possible_ranks:
        raise ValueError(f"Rank {rank} not available")

    
    lineage_dict = {}
    sample_name_dict = {}
    seen_lineages = set()

    # create dictionary that holds all of the sample names
    for file in csvs:
        sample_name = file.split('.')[0]
        sample_name_dict[sample_name] = 0 

    for file in csvs:
        with open(file, 'r') as fp:
            r = csv.DictReader(fp)
            for n, row in enumerate(r):
                if row["rank"] == rank:
                    seen_lineages.add(row["lineage"])
            fp.close()

    for lineage in seen_lineages:
        lineage_dict[lineage] = sample_name_dict.copy()

    for sample in sample_name_dict:
        with open(sample + ".csv", "r") as fp:
            r = csv.DictReader(fp)
            for n, row in enumerate(r):
                if row["rank"] == rank:
                    lineage = (row["lineage"])
                    fraction = (row["fraction"])
                    lineage_dict[lineage][sample] = fraction
            fp.close()


    samples.insert(0, "lineage")
    with open(output_csv, 'w') as f_output:
        w = csv.DictWriter(f_output, samples)
        for key,val in sorted(lineage_dict.items()):
            row = {'lineage': key}
            row.update(val + 1)
            w.writerow(row)

In [16]:
format_tax_to_frac(csvs, "species", "tmp.csv")

TypeError: __init__() missing 1 required positional argument: 'fieldnames'